In [20]:
import pandas as pd
from pathlib import Path

import sys
sys.path.append('../src')
from utils.duckdb_utils import connect_duckdb
from utils.duckdb_utils import run_sql_file
# import utils.site_list as sl

In [23]:
with connect_duckdb() as con:
    tables = con.execute("SHOW TABLES").fetchall()
    table_names = [table[0]for table in tables]

    print(table_names)

['daily_observations', 'parameter', 'site', 'site_parameter', 'vw_nwis_annual_stats_local', 'vw_nwis_daily_stats_local', 'vw_nwis_iv_local']


In [33]:
query = """
    SELECT hydro_area_nm, site_nm, year, max_value, max_date
    FROM vw_nwis_annual_stats_local
    WHERE parameter_cd = '00060'
    order by year DESC, hydro_area_nm, site_nm
    """

with connect_duckdb() as con:
    result = con.execute(query).df()
    print(result.head())

    hydro_area_nm                                            site_nm  year  \
0  Colorado River  COLO RIVER BELOW GRAND VALLEY DIV NR PALISADE, CO  2025   
1  Colorado River  COLORADO R. ABV GUNNISON R. AT GRAND JUNCTION, CO  2025   
2  Colorado River      COLORADO RIVER AT GYPSUM CANYON NEAR HITE, UT  2025   
3  Colorado River                     COLORADO RIVER NEAR CAMEO, CO.  2025   
4  Colorado River       COLORADO RIVER NEAR COLORADO-UTAH STATE LINE  2025   

   max_value   max_date  
0     6640.0 2025-06-04  
1     6870.0 2025-06-04  
2    19100.0 2025-06-06  
3     7970.0 2025-06-04  
4    11900.0 2025-06-04  


In [4]:
query = "SELECT * FROM site"
query = "SELECT * FROM parameter"

In [22]:
run_sql_file(Path('../db/views.sql'))

In [12]:
with connect_duckdb() as con:
    result = con.execute(query).df()
print(result.head())

   parameter_id parameter_cd                 parameter_nm parameter_dsc  \
0             1           60                    Discharge          None   
1             2           10            Water temperature          None   
2             3        62614  Reservoir surface elevation          None   
3             4           15                 Bank storage          None   
4             5           29                       Inflow          None   

  unit_cd                unit_nm                  create_ts  \
0     cfs  Cubic feet per second 2025-06-17 17:28:46.429526   
1       C        Degrees Celsius 2025-06-17 17:28:46.429526   
2      ft                   Feet 2025-06-17 17:28:46.429526   
3      af              Acre-feet 2025-06-17 17:28:46.429526   
4     cfs  Cubic feet per second 2025-06-17 17:28:46.429526   

                   update_ts  
0 2025-06-17 17:28:46.429880  
1 2025-06-17 17:28:46.429880  
2 2025-06-17 17:28:46.429880  
3 2025-06-17 17:28:46.429880  
4 2025-06-17 17

In [11]:
param = pd.read_csv('../artifacts/parameters.csv')

with connect_duckdb() as con:
    con.register('param', param)
    con.execute('INSERT INTO parameter SELECT * FROM param')